# Índice

1. **Introducción**
   a. Explicaciones visuales en modelos de redes profundas  
2. **Antecedentes**  
   a. Oclussion Maps  
   b. Guided Backpropagation  
   c. CAM  
3. **GradCAM**  
   a. Explicación  
   b. Ejemplos clasificación: ResNet50, VGG16, etc.  
   c. Ejemplos otras tareas: Automatic Captioning  
   d. Ventajas y desventajas  
4. **Mejoras: GradCAM++ y otros métodos**

___

# 1. Introducción

La explicabilidad en redes neuronales convolucionales (CNN) ha surgido como un aspecto clave en el campo del reconocimiento de patrones, donde no solo interesa alcanzar un alto desempeño, sino también comprender qué información utiliza la red para tomar decisiones. Un primer avance fue **Class Activation Mapping (CAM, 2016)**, que permitió visualizar las regiones más relevantes de una imagen para la clasificación, aunque con la limitación de aplicarse únicamente a arquitecturas específicas con *global average pooling*. Más adelante, **Grad-CAM (2017)** amplió esta idea utilizando gradientes retropropagados, logrando un método versátil aplicable a una amplia variedad de arquitecturas y consolidándose como la técnica de referencia en explicabilidad. A partir de allí se desarrollaron extensiones como **Grad-CAM++ (2018)**, que mejora la localización en escenarios complejos, y **Score-CAM (2019)**, que evita depender directamente de los gradientes. Sin embargo, el salto conceptual decisivo que abrió la puerta al estudio moderno de la explicabilidad fue el de Grad-CAM.

# 2. Antecedentes

MINI TEXTO

### Occlusion Maps

**Occlusion Maps** es uno de los métodos más intuitivos y directos para generar explicaciones visuales en redes neuronales. La idea central consiste en evaluar la sensibilidad de la predicción de la red cuando partes de la entrada son ocultadas o reemplazadas por un valor neutro (por ejemplo, un parche negro o ruido). En la práctica, se recorre la imagen aplicando una “ventana de oclusión” que cubre sistemáticamente diferentes regiones, y se mide cómo varía la probabilidad asociada a la clase objetivo. Las zonas donde la predicción se ve más afectada se interpretan como las más relevantes para la decisión del modelo.  

Este enfoque tiene varias ventajas: es sencillo de implementar, no depende de modificaciones en la arquitectura del modelo ni de los gradientes, y produce mapas de importancia fáciles de interpretar. No obstante, también presenta desventajas: es computacionalmente costoso (ya que requiere múltiples inferencias por imagen), la resolución depende del tamaño del parche usado, y no captura bien interacciones complejas entre diferentes regiones de la entrada. A pesar de estas limitaciones, Occlusion Maps es un método de referencia en la literatura y se utiliza frecuentemente como baseline en estudios de explicabilidad.  

**Referencias:**  
- Zeiler, M. D., & Fergus, R. (2014). *Visualizing and Understanding Convolutional Networks*. In ECCV. Springer.  
- Samek, W., Montavon, G., Vedaldi, A., Hansen, L. K., & Müller, K. R. (Eds.). (2019). *Explainable AI: Interpreting, Explaining and Visualizing Deep Learning*. Springer.  


### Guided Backpropagation

**Guided Backpropagation** es un método de visualización que busca identificar qué regiones de una imagen influyen positivamente en la activación de una clase objetivo dentro de una red neuronal profunda. A diferencia del backpropagation estándar, este enfoque modifica el flujo de gradientes en las funciones de activación ReLU, permitiendo únicamente el paso de aquellos gradientes positivos asociados a activaciones positivas. El resultado es un mapa de sensibilidad de alta resolución que resalta bordes y detalles finos de la entrada, mostrando qué píxeles contribuyen a la predicción. Sin embargo, no es estrictamente clase-específico, ya que puede resaltar patrones que activan la red independientemente de la salida seleccionada. Por este motivo, suele utilizarse en combinación con Grad-CAM, logrando visualizaciones más interpretables y centradas en la clase de interés.  

**Referencias:**  
- Springenberg, J. T., Dosovitskiy, A., Brox, T., & Riedmiller, M. (2014). *Striving for Simplicity: The All Convolutional Net*. arXiv:1412.6806.  
- Montavon, G., Samek, W., & Müller, K. R. (2018). *Methods for interpreting and understanding deep neural networks*. Digital Signal Processing, 73, 1–15.  


### CAM

**CAM** fue introducido por Zhou et al. (2016) y se basa en la utilización de una capa de *global average pooling (GAP)* antes de la capa final de clasificación. La idea es que los pesos de la capa de salida pueden emplearse para ponderar los mapas de activación de la última capa convolucional, generando así un mapa de calor que indica las regiones de la imagen más relevantes para una clase determinada.

Se contruyen los mapas de activacion para cada clase de interes como:

$$
M_c(x,y) = \sum_k w_k^c f_k(x,y)
$$

donde:

* $M_c(x,y)$ es el mapa de activación para la clase $c$,
* $f_k(x,y)$ es el mapa de activación de la característica $k$,
* $w_k^c$ es el peso asociado a la clase $c$ en la capa de salida.


Además de requerir global average pooling, CAM solo puede aplicarse a capas cuya ultima capa sea una red densa.
Un uso interesante es la posibilidad de hacer localizacion de objetos mediante una red que no haya sido entrenada con bounding boxes http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf

**Referencia:**
Zhou, B., Khosla, A., Lapedriza, A., Oliva, A., & Torralba, A. (2016). *Learning Deep Features for Discriminative Localization*. CVPR.


# 3. Grad-CAM


### Metodología

**(CONVERTIR PRINCIPALMENTE EN UN PARRAFO - AGREGAR UNA INTUICIÓN)**
Grad-CAM es una técnica que permite visualizar las regiones de una imagen que son relevantes para la decisión de un modelo de red neuronal. A través del uso de gradientes, se puede obtener un mapa de calor que indica qué partes de la imagen contribuyen más a la activación de una clase específica. Esto se logra calculando los gradientes de la clase de interés respecto a los mapas de activación de una capa convolucional, lo que permite ponderar las características de acuerdo con su importancia para la predicción.

**Aporte:**
Grad-CAM, propuesto por Selvaraju et al. (2017), generaliza la idea de CAM para que pueda aplicarse a una gran variedad de arquitecturas. Utiliza los gradientes de la clase de interés respecto a los mapas de activación de una capa convolucional, ponderando las características de acuerdo con su importancia para la predicción.

**Fórmula principal:**

1. Calcular los gradientes de la puntuación de la clase $y^c$ respecto a los mapas de activación $A^k$:

$$
\alpha_k^c = \frac{1}{Z} \sum_i \sum_j \frac{\partial y^c}{\partial A_{ij}^k}
$$

2. Generar el mapa de activación como:

$$
L_{\text{Grad-CAM}}^c = ReLU\left(\sum_k \alpha_k^c A^k \right)
$$

donde $ReLU$ asegura que solo se conserven las contribuciones positivas.

**Ventajas:**

* Se puede aplicar a múltiples arquitecturas (sin necesidad de GAP).
* Permite obtener explicaciones visuales interpretables en tareas de clasificación, detección y segmentación.

**Limitaciones:**

* La resolución espacial del mapa depende de la capa elegida (generalmente baja).
* Tiende a resaltar regiones amplias sin un detalle fino de contornos.

**Referencia:**
Selvaraju, R. R., et al. (2017). *Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization*. ICCV.

#### Ejemplo 1 de Grad-CAM:clasificacion de imagenes con ResNet50 

**Figura 1**
Ejemplo con un modelo y gradcam, varias instancias y varias clases.

#### Ejemplo 2 de Grad-CAM: XXXX

**Figura 2**: Segmentación/captioning/etc


#### Ejemplo 3 de varias arquitecturas

**Figura 3** Ejemplo de varias arquitecturas

                    modelo1                    modelo2
            cat1|cat2  perro1|perro2    cat1|cat2  perro1|perro2

instancia1

instancia2

instancia3

instancia4

# 4. Más allá de Grad-CAM

Con la intención de mejorar la definición y la estabilidad de los mapas de calor explicativos, se han propuesto diversas variantes de Grad-CAM. Estas buscan refinar la localización, reducir la dependencia de los gradientes o aumentar la resolución espacial de las visualizaciones.

### Grad-CAM++ (2018)

* **Idea:** ajusta la forma de calcular los pesos $\alpha_k^c$ para tener en cuenta múltiples instancias de un mismo objeto en la imagen.
* **Pro:** produce mapas más precisos en escenarios complejos y con objetos superpuestos.
* **Con:** mayor costo computacional y complejidad en el cálculo.

#### Comparación de métodos de explicabilidad basados en Grad-CAM

**Figura 4**: Comparación de métodos de explicabilidad basados en Grad-CAM